In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from commando import ComManDo
from commando.evaluation import test_partial
import numpy as np
from sklearn.metrics import confusion_matrix
import torch

2022-01-18 18:45:31.864013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 18:45:31.864044: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data1 = np.loadtxt("../data/Unioncom/MMD/s1_mapped1.txt")
data2 = np.loadtxt("../data/Unioncom/MMD/s1_mapped2.txt")
num_aligned = int(.8 * len(data2))
data2, data2_withheld = data2[:num_aligned], data2[num_aligned:]

# Predict Missing Modality

In [4]:
# Align data
cm = ComManDo(aligned_idx=2*(range(num_aligned),))
cm_data = cm.fit_transform(dataset=[data1, data2])

use random seed: 666
Shape of Raw data
Dataset 0: (300, 2000)
Dataset 1: (240, 1000)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[100/2000] err:2.7339 alpha:0.4663
epoch:[200/2000] err:2.2796 alpha:0.4117
epoch:[300/2000] err:2.1866 alpha:0.4259
epoch:[400/2000] err:2.1304 alpha:0.4381
epoch:[500/2000] err:1.9112 alpha:0.4565
epoch:[600/2000] err:1.5409 alpha:0.4822
epoch:[700/2000] err:1.3358 alpha:0.5007
epoch:[800/2000] err:1.1939 alpha:0.5133
epoch:[900/2000] err:1.0500 alpha:0.5237
epoch:[1000/2000] err:0.9691 alpha:0.5316
epoch:[1100/2000] err:0.9301 alpha:0.5373
epoch:[1200/2000] err:0.9055 alpha:0.5417
epoch:[1300/2000] err:0.8871 alpha:0.5454
epoch:[1400/2000] err:0.8722 alpha:0.5487
epoch:[1500/2000] err:0.8598 alpha:0.5516
epoch:[1600/2000] err:0.8493 alpha:0.5542
epoch:[1700/2000] err:0.8403 alpha:0.5566
epoch:[1800/2000] err:0.8326 alpha:0.5588
epoch:[1900/2000] err:0.8258 alpha:0.5607
epoch:[2000/2000] err:0.8198

In [5]:
# Compile prediction and true
predicted = cm.model.decoders[1](torch.tensor(cm_data[0][num_aligned:])).detach().cpu().numpy()
predicted = predicted.flatten() > .5
actual = (data2_withheld - np.min(data2_withheld)) / (np.max(data2_withheld) - np.min(data2_withheld))
actual = actual.flatten() > .5

In [6]:
# Produce confusion matrix
# tn fp
# fn tp
confusion_matrix(actual, predicted, labels=(False, True), normalize='true')

array([[0.6577473 , 0.3422527 ],
       [0.23421089, 0.76578911]])